The Unreasonable Effectiveness of Recurrent Neural Networks
----

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

## RNN computation.

So how do these things work? At the core, RNNs have a deceptively simple API: They accept an input vector x and give you an output vector y. However, crucially this output vector’s contents are influenced not only by the input you just fed in, but also on the entire history of inputs you’ve fed in in the past. Written as a class, the RNN’s API consists of a single step function:

**RNN计算** 这些事情是怎么工作的？本质上讲，RNNs的API简单得不可思议：它们接收一个输入向量x，然后输出一个向量y。非常重要的是，输出向量的内容不仅仅取决于输入，而且来依赖于以前所有的输入。如果写成类的话，RNN的接口就是一个step函数：

In [ ]:
runn = RNN()
y = rnn.step(x)  # x是输入向量，y是输出向量

The RNN class has some internal state that it gets to update every time step is called. In the simplest case this state consists of a single hidden vector h. Here is an implementation of the step function in a Vanilla RNN:

RNN类有保存了一些内部状态，每次step函数调用的时候，会更新这些状态。一种最简单的情况下，内部状态用一个隐藏向量h来表示。下面是一个普通的RNN的step函数的实现：

In [ ]:
class RNN:
    # ...
    def step(self, x):
        # 更新隐藏的状态
        self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
        # 计算输出向量
        y = np.dot(self.W_hy, self.h)
        return y

The above specifies the forward pass of a vanilla RNN. This RNN’s parameters are the three matrices W_hh, W_xh, W_hy. The hidden state self.h is initialized with the zero vector. The np.tanh function implements a non-linearity that squashes the activations to the range [-1, 1]. Notice briefly how this works: There are two terms inside of the tanh: one is based on the previous hidden state and one is based on the current input. In numpy np.dot is matrix multiplication. The two intermediates interact with addition, and then get squashed by the tanh into the new state vector. If you’re more comfortable with math notation, we can also write the hidden state update as ht=tanh(Whhht−1+Wxhxt)ht=tanh⁡(Whhht−1+Wxhxt), where tanh is applied elementwise.

上面就是一个平凡的RNN正向计算的过程。这个RNN的参数就是三个矩阵 `W_hh`，`W_xh` 和 `W_hy`。隐藏状态 `self.h`初始化为0. `np.tanh`是一个非线性激励函数输出的范围是 `[-1, 1]`。简单来看，在tanh里有两个方面：一个基于之前的隐藏状态，另一个基于当前的输入。`np.dot`是矩阵乘法。把这两个中间状态相加，然后传给tanh得到一个新的状态。如果你对数学公式更加习惯的话，隐藏状态的更新可以表示为这个公式：$ h_t = tanh( W_{hh} h_{t-1} + W_{xh} x_t) $ 其中tanh对向量的每一位按位进行计算的。
 
We initialize the matrices of the RNN with random numbers and the bulk of work during training goes into finding the matrices that give rise to desirable behavior, as measured with some loss function that expresses your preference to what kinds of outputs y you’d like to see in response to your input sequences x.

我们把RNN的这几个矩阵初始化为随机数，然后开始训练去找到一种期望的行为。这种行为由某种损失函数来度量，它表达了我们在给定了输入序列 x 后希望看到什么样的输出 y。

## Going deep.

RNNs are neural networks and everything works monotonically better (if done right) if you put on your deep learning hat and start stacking models up like pancakes. For instance, we can form a 2-layer recurrent network as follows:

RNNs也是神经网络，当你把深度学习中的方法运用到RNN上的时候，也同样有效（如果实现得对的话）。例如，我们可以创建一个两层的循环神经网络：

In [ ]:
y1 = rnn1.step(x)
y = rnn2.step(y1)

In other words we have two separate RNNs: One RNN is receiving the input vectors and the second RNN is receiving the output of the first RNN as its input. Except neither of these RNNs know or care - it’s all just vectors coming in and going out, and some gradients flowing through each module during backpropagation.

换句话说，我们有两个单独的RNN。一个用来接收输入向量，另一个接收第一个RNN的输出当成输入。这两个RNN互相不知道有对方而且也不关心。只是向量作为输入，向量作为输出，而且在反向传递的时候梯度会传递到每个模块。

## Getting fancy. 

I’d like to briefly mention that in practice most of us use a slightly different formulation than what I presented above called a Long Short-Term Memory (LSTM) network. The LSTM is a particular type of recurrent network that works slightly better in practice, owing to its more powerful update equation and some appealing backpropagation dynamics. I won’t go into details, but everything I’ve said about RNNs stays exactly the same, except the mathematical form for computing the update (the line self.h = ... ) gets a little more complicated. From here on I will use the terms “RNN/LSTM” interchangeably but all experiments in this post use an LSTM.

在实践中，我们使用一个和上面稍有区别的公式，叫场短时记忆网络（LSTM）。LSTM是一种特殊类型的RNN，在实际中往往效果更好，这得益于它的更新状态和应用梯度的巧妙方式。我不会深入LSTM的细节，但是前面说的所有跟RNN相关的知识对于LSTM都是一样的，只是在更新隐藏状态的公式更复杂一些而已。从现在开始我会交换使用RNN和LSTM，但是实际上工作里用的都是LSTM。

# Character-Level Language Models

Okay, so we have an idea about what RNNs are, why they are super exciting, and how they work. We’ll now ground this in a fun application: We’ll train RNN character-level language models. That is, we’ll give the RNN a huge chunk of text and ask it to model the probability distribution of the next character in the sequence given a sequence of previous characters. This will then allow us to generate new text one character at a time.

字符水平的语言模型。我们给RNN一大堆文本输入然后让它来学习得到一个分布，这个分布根据序列的头几个字符来给出序列中的下一个字符。这样我们就通过每次生成一个字符来生成一段文本。

As a working example, suppose we only had a vocabulary of four possible letters “helo”, and wanted to train an RNN on the training sequence “hello”. This training sequence is in fact a source of 4 separate training examples: 1. The probability of “e” should be likely given the context of “h”, 2. “l” should be likely in the context of “he”, 3. “l” should also be likely given the context of “hel”, and finally 4. “o” should be likely given the context of “hell”.

假设我们有一个四个字符的字母表“helo”，而且我们希望训练一个RNN在序列“hello”上。这个训练序列实际上是四个单独的训练样本。
1. 在给定了上下文为“h“的时候，输出为”e“的概率。
2. 在给定了上下文为“he”的时候，输出为“l"
3. 上下文“hel”，输出“l”
4. 上下文“hell”，输出为“o“

Concretely, we will encode each character into a vector using 1-of-k encoding (i.e. all zero except for a single one at the index of the character in the vocabulary), and feed them into the RNN one at a time with the step function. We will then observe a sequence of 4-dimensional output vectors (one dimension per character), which we interpret as the confidence the RNN currently assigns to each character coming next in the sequence. Here’s a diagram:

具体来说，我们把每个字符编码成一个1-of-k的向量。（除了字符对应的下标为1，其他位置为0）。然后把他们依次输入到RNN的step函数，我们来观察输出的序列，它解释为RNN给出的下一个字符的信心。

![RNN](http://karpathy.github.io/assets/rnn/charseq.jpeg)

An example RNN with 4-dimensional input and output layers, and a hidden layer of 3 units (neurons). This diagram shows the activations in the forward pass when the RNN is fed the characters "hell" as input. The output layer contains confidences the RNN assigns for the next character (vocabulary is "h,e,l,o"); We want the green numbers to be high and red numbers to be low.

一个RNN的例子，输入4维，输出4维，隐藏层有3个单元（神经元）。图显示了在输入为”hell“的时候，正向传递给出的激励。输出层包含了RNN对下一个字符的信心（字母表为”h,e,l,o“）。我们希望绿色的数字变高，红色的数字变低。

For example, we see that in the first time step when the RNN saw the character “h” it assigned confidence of 1.0 to the next letter being “h”, 2.2 to letter “e”, -3.0 to “l”, and 4.1 to “o”. Since in our training data (the string “hello”) the next correct character is “e”, we would like to increase its confidence (green) and decrease the confidence of all other letters (red). Similarly, we have a desired target character at every one of the 4 time steps that we’d like the network to assign a greater confidence to. Since the RNN consists entirely of differentiable operations we can run the backpropagation algorithm (this is just a recursive application of the chain rule from calculus) to figure out in what direction we should adjust every one of its weights to increase the scores of the correct targets (green bold numbers). We can then perform a parameter update, which nudges every weight a tiny amount in this gradient direction. If we were to feed the same inputs to the RNN after the parameter update we would find that the scores of the correct characters (e.g. “e” in the first time step) would be slightly higher (e.g. 2.3 instead of 2.2), and the scores of incorrect characters would be slightly lower. We then repeat this process over and over many times until the network converges and its predictions are eventually consistent with the training data in that correct characters are always predicted next.

例如，第一次step，RNN看到了字符”h”，它给下一次字符“h”的信心是1.0，“e”是2.2，“l”是-3.0，“o”是4.1.因为在我们的训练数据里，下一个字符是“e”，我们希望增加它的信心（绿色）减低其他字符的信息（红色）。类似的，4次step我们都有期望增大其信心的字符。因为RNN全部都有可导的操作组成，我们可以使用反向传递算法来确定要更新的方向。